# Création de prompts pour le nommage et la description des clusters

In [2]:
Sys.setlocale("LC_ALL", "fr_CA.UTF-8")

# Library
library(dplyr)
library(tidyr)
library(ggplot2)
library(ggtext)
library(tidytext)
library(ellipsellm)

# Load Data and Clustering Results
kmeans_result <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_kmeans.rds") # nolint
dfUsedForClustering <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_2025.rds")
df_clusters_composition <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_clusters_composition.rds")

[1] "fr_CA.UTF-8/fr_CA.UTF-8/fr_CA.UTF-8/C/fr_CA.UTF-8/C"

In [11]:
cluster_names <- unique(dfUsedForClustering$cluster_name)
cluster_names <- cluster_names[order(cluster_names)]

prompts <- list()

for (cluster_value in cluster_names) {
  cluster_data <- dfUsedForClustering %>% filter(cluster_name == cluster_value)
  cluster_vars <- df_clusters_composition %>% filter(cluster_name == cluster_value)

  # Calculer quelques statistiques démographiques pour le cluster
  perc_hommes     <- mean(cluster_data$ses_genderMale, na.rm = TRUE) * 100
  perc_immigrants <- mean(cluster_data$ses_immigrant, na.rm = TRUE) * 100
  perc_langEn     <- mean(cluster_data$ses_languageEnglish, na.rm = TRUE) * 100
  perc_langFr     <- mean(cluster_data$ses_languageFrench, na.rm = TRUE) * 100
  perc_ses_languageOther <- mean(cluster_data$ses_languageOther, na.rm = TRUE) * 100
  perc_ethn_white <- mean(cluster_data$ses_ethnicityWhite, na.rm = TRUE) * 100
  perc_hetero     <- mean(cluster_data$ses_sexOrientationHetero, na.rm = TRUE) * 100
  educ_mean_BHS       <- mean(cluster_data$ses_educBHS, na.rm = TRUE) * 100
  educ_mean_PostHS       <- mean(cluster_data$ses_educPostHS, na.rm = TRUE) * 100
  educ_mean_Univ       <- mean(cluster_data$ses_educUniv, na.rm = TRUE) * 100
  age_mean        <- mean(cluster_data$ses_age, na.rm = TRUE) * 80
  revenu_moyen_Low    <- mean(cluster_data$ses_incomeLow, na.rm = TRUE) * 100
  revenu_moyen_Mid    <- mean(cluster_data$ses_incomeMid, na.rm = TRUE) * 100
  revenu_moyen_High    <- mean(cluster_data$ses_incomeHigh, na.rm = TRUE) * 100

  # Génération du prompt de suggestion de noms
  prompt_description <- (paste0(
  "Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona ", cluster_value, " :\n\n",
  
  "Variables avec une moyenne significativement plus élevée que la moyenne de la population en général :\n",
  if (any(cluster_vars$color == "green")) {
    paste(cluster_vars[cluster_vars$color == "green", "variable"], collapse = "\n")
  } else {
    "Aucune variable n'a une moyenne significativement plus élevée que la moyenne de la population en général."
  },
  
  "\n\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\n",
  if (any(cluster_vars$color == "red")) {
    paste(cluster_vars[cluster_vars$color == "red", "variable"], collapse = "\n")
  } else {
    "Aucune variable avec une moyenne significativement plus basse que la moyenne de la population en général."
  },
  
  "\n\nCe cluster est composé de :\n", 
  "Hommes: ", round(perc_hommes, 1), "%, ",
  "Immigrants: ", round(perc_immigrants, 1), "%\n",
  "Âge: ", round(age_mean, 1), " ans\n",
  "Revenus:\n",
  "  Revenu Bas: ", round(revenu_moyen_Low, 1), " %\n",
  "  Revenu Mid: ", round(revenu_moyen_Mid, 1), " %\n",
  "  Revenu High: ", round(revenu_moyen_High, 1), " %\n",
  "Langues:\n",
  "  En: ", round(perc_langEn, 1), "%, ",
  "Fr: ", round(perc_langFr, 1), "%, ",
  "Autres: ", round(perc_ses_languageOther, 1), "%\n",
  "EducBHS: ", round(educ_mean_BHS, 1), "%\n",
  "EducPostHS: ", round(educ_mean_PostHS, 1), "%\n",
  "EducUniv: ", round(educ_mean_Univ, 1), "%\n",
  "Ethnie (Blanc): ", round(perc_ethn_white, 1), "%\n",
  "Orientation (Hétéro): ", round(perc_hetero, 1), "%\n"
))

  prompts[[cluster_value]] <- prompt_description
}


In [4]:
prompts

$`1`
[1] "Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 1 :\n\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nc(\"dv_voteChoiceLPC\", \"ses_educUniv\", \"ses_regionOntario\")\n\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nc(\"dv_voteChoiceBQ\", \"dv_voteChoiceCPC\", \"dv_voteChoiceNDP\", \"dv_voteChoiceOther\", \"ses_languageFrench\", \"ses_regionQuebec\", \"ses_regionWest\")\n\nCe cluster est composé de :\nHommes: 46.9%, Immigrants: 23.5%\nÂge: 51.9 ans\nRevenus:\n  Revenu Bas: 12.3 %\n  Revenu Mid: 72.8 %\n  Revenu High: 14.8 %\nLangues:\n  En: 93.8%, Fr: 1.2%, Autres: 4.9%\nEducBHS: 16%\nEducPostHS: 23.5%\nEducUniv: 60.5%\nEthnie (Blanc): 69.1%\nOrientation (Hétéro): 79%\n"

$`10`
[1] "Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 10 :\n\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nses_regionAtlantic\n\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nc(\"dv_voteChoiceBQ\", \"ses_educUniv\", \"ses_languageFrench\", \"ses_regionOntario\", \"ses_regionQuebec\", \"ses_regionWest\")\n\nCe cluster est composé de :\nHommes: 59.5%, Immigrants: 18.9%\nÂge: 53 ans\nRevenus:\n  Revenu Bas: 13.5 %\n  Revenu Mid: 81.1 %\n  Revenu High: 5.4 %\nLangues:\n  En: 100%, Fr: 0%, Autres: 0%\nEducBHS: 29.7%\nEducPostHS: 40.5%\nEducUniv: 29.7%\nEthnie (Blanc): 94.6%\nOrientation (Hétéro): 78.4%\n"

$`11`
[1] "Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 11 :\n\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nc(\"dv_voteChoiceLPC\", \"ses_regionWest\")\n\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nc(\"dv_voteChoiceBQ\", \"dv_voteChoiceCPC\", \"dv_voteChoiceNDP\", \"dv_voteChoiceOther\", \"ses_languageFrench\", \"ses_regionOntario\", \"ses_regionQuebec\")\n\nCe cluster est composé de :\nHommes: 50%, Immigrants: 15.2%\nÂge: 44.8 ans\nRevenus:\n  Revenu Bas: 10.6 %\n  Revenu Mid: 80.3 %\n  Revenu High: 9.1 %\nLangues:\n  En: 93.9%, Fr: 1.5%, Autres: 4.5%\nEducBHS: 18.2%\nEducPostHS: 31.8%\nEducUniv: 50%\nEthnie (Blanc): 74.2%\nOrientation (Hétéro): 92.4%\n"

$`12`
[1] "Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 12 :\n\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nc(\"dv_solidity\", \"dv_turnout\", \"dv_voteChoiceCPC\", \"ses_regionWest\")\n\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nc(\"dv_voteChoiceBQ\", \"dv_voteChoiceLPC\", \"dv_voteChoiceNDP\", \"dv_voteChoiceOther\", \"ses_incomeLow\", \"ses_languageFrench\", \"ses_regionOntario\", \"ses_regionQuebec\")\n\nCe cluster est composé de :\nHommes: 64.1%, Immigrants: 16.5%\nÂge: 45.4 ans\nRevenus:\n  Revenu Bas: 6.8 %\n  Revenu Mid: 78.6 %\n  Revenu High: 14.6 %\nLangues:\n  En: 97.1%, Fr: 0%, Autres: 2.9%\nEducBHS: 25.2%\nEducPostHS: 35%\nEducUniv: 39.8%\nEthnie (Blanc): 77.7%\nOrientation (Hétéro): 94.2%\n"

$`2`
[1] "Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 2 :\n\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\ndv_voteChoiceGPC\n\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nc(\"dv_solidity\", \"dv_voteChoiceBQ\", \"dv_voteChoiceCPC\", \"dv_voteChoiceLPC\", \"dv_voteChoiceNDP\", \"dv_voteChoiceOther\")\n\nCe cluster est composé de :\nHommes: 54.1%, Immigrants: 27%\nÂge: 41.1 ans\nRevenus:\n  Revenu Bas: 21.6 %\n  Revenu Mid: 70.3 %\n  Revenu High: 8.1 %\n

In [5]:
reponses_openai <- list()
# Afficher ou traiter tous les prompts
for (cluster in names(prompts)) {
  # Par exemple, envoyer chaque prompt à OpenAI (si la fonction est correctement définie)
  conv <- openai_create_conversation(prompts[[cluster]])
  resp <- openai_chat_completion(conv)
  reponses_openai[[cluster]] <- resp$content
}


In [6]:
reponses_openai

$`1`
[1] "Pour une personne habitant au Canada correspondant à ce persona, je propose le prénom \"Andrew\". Voici une brève description du persona :\n\n**Andrew** est un homme de 52 ans vivant en Ontario, au Canada. Il est anglophone, avec une éducation universitaire, ce qui le place dans la catégorie des personnes ayant un niveau d'éducation plus élevé que la moyenne. Andrew est probablement engagé professionnellement dans un secteur qui valorise l'éducation postsecondaire, et il appartient à la classe de revenu moyen. Il montre une préférence marquée pour le Parti libéral du Canada (LPC) lorsqu'il vote, et il est peu susceptible de soutenir d'autres partis tels que le Bloc Québécois, le Parti conservateur, ou le Nouveau Parti démocratique.\n\nAndrew n'est pas un immigrant, mais il partage son environnement social avec une minorité importante d'immigrants, ce qui pourrait influencer ses perspectives et ses interactions quotidiennes. En tant que Canadien anglophone vivant en Ontario, il n’a pas beaucoup d'interactions avec la culture francophone ou les régions à l'ouest du Canada.\n\nSocialement, il est principalement entouré de personnes de son groupe ethnique, mais il est ouvert à la diversité culturelle, étant donné la composition démographique de sa région. Andrew est hétérosexuel et vit probablement dans une famille traditionnelle ou dans un couple, ce qui cadre avec la majorité des personnes de son orientation dans sa tranche d'âge et son environnement."

$`10`
[1] "Pour une personne vivant au Canada, et plus précisément dans la région atlantique, un prénom qui serait à la fois anglophone et courant pourrait être \"James\". Voici une brève description pour le persona 10 :\n\n**Nom :** James\n\n**Âge :** 53 ans\n\n**Région :** Atlantique du Canada\n\n**Profil démographique :**\n- **Genre :** Homme\n- **Immigrant :** Non (compte tenu du pourcentage faible d'immigrants dans ce cluster)\n- **Langue :** Parle uniquement anglais\n- **Éducation :** A terminé des études postsecondaires, mais pas nécessairement universitaires\n- **Revenu :** Principalement moyen, avec peu de probabilité d'être dans la tranche de revenu élevé\n- **Origine ethnique :** Majoritairement blanc\n- **Orientation sexuelle :** Principalement hétérosexuel\n\n**Caractéristiques supplémentaires :**\n- James a grandi et vit toujours dans la région atlantique du Canada, ce qui influence ses habitudes et ses préférences culturelles.\n- Il n'a pas tendance à soutenir le Bloc Québécois, ce qui est cohérent avec son profil de résidant anglophone hors du Québec.\n- Bien qu'il ait poursuivi des études postsecondaires, il n'a pas nécessairement un diplôme universitaire, ce qui est typique de son groupe démographique.\n- Sa vie est principalement basée autour de sa communauté locale, et il est fier de son héritage et de sa culture régionale.\n\nJames est typiquement engagé dans sa communauté, apprécie les traditions locales, et est généralement satisfait de son mode de vie dans la région atlantique du Canada."

$`11`
[1] "Pour une personne habitant au Canada correspondant à ce profil, je propose le prénom \"Ethan\". Ce prénom est courant et populaire dans les régions anglophones du Canada, ce qui correspond bien au fait que 93.9% de ce groupe parle anglais.\n\n**Description du persona 11 :**\n\nEthan est un homme de 44 ans qui vit dans l'Ouest du Canada. Il est anglophone et a une prédilection pour le Parti Libéral du Canada (LPC). Il a fait des études universitaires, ce qui est cohérent avec le fait que 50% des personnes de ce groupe ont un diplôme universitaire. Ethan est probablement d'origine ethnique blanche, comme 74.2% de son groupe, et il est hétérosexuel.\n\nEthan n'est pas immigrant, mais il est ouvert et tolérant, vivant dans une région avec une certaine diversité. Il a un revenu moyen, ce qui est représentatif de 80.3% de son groupe. Bien qu'il connaisse quelques mots de français, il n'est pas francophone, et il vit probablement loin des régions principale

In [7]:
description_clusters <- data.frame(prompt = unlist(prompts), description = unlist(reponses_openai))

In [8]:
description_clusters

,prompt,description
,<chr>,<chr>
1,"Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 1 : Variables avec une moyenne significativement plus élevée que la moyenne de la population en général : c(""dv_voteChoiceLPC"", ""ses_educUniv"", ""ses_regionOntario"") Variables avec une moyenne significativement plus basse que la moyenne de la population en général : c(""dv_voteChoiceBQ"", ""dv_voteChoiceCPC"", ""dv_voteChoiceNDP"", ""dv_voteChoiceOther"", ""ses_languageFrench"", ""ses_regionQuebec"", ""ses_regionWest"") Ce cluster est composé de : Hommes: 46.9%, Immigrants: 23.5% Âge: 51.9 ans Revenus: Revenu Bas: 12.3 % Revenu Mid: 72.8 % Revenu High: 14.8 % Langues: En: 93.8%, Fr: 1.2%, Autres: 4.9% EducBHS: 16% EducPostHS: 23.5% EducUniv: 60.5% Ethnie (Blanc): 69.1% Orientation (Hétéro): 79%","Pour une personne habitant au Canada correspondant à ce persona, je propose le prénom ""Andrew"". Voici une brève description du persona : **Andrew** est un homme de 52 ans vivant en Ontario, au Canada. Il est anglophone, avec une éducation universitaire, ce qui le place dans la catégorie des personnes ayant un niveau d'éducation plus élevé que la moyenne. Andrew est probablement engagé professionnellement dans un secteur qui valorise l'éducation postsecondaire, et il appartient à la classe de revenu moyen. Il montre une préférence marquée pour le Parti libéral du Canada (LPC) lorsqu'il vote, et il est peu susceptible de soutenir d'autres partis tels que le Bloc Québécois, le Parti conservateur, ou le Nouveau Parti démocratique. Andrew n'est pas un immigrant, mais il partage son environnement social avec une minorité importante d'immigrants, ce qui pourrait influencer ses perspectives et ses interactions quotidiennes. En tant que Canadien anglophone vivant en Ontario, il n’a pas beaucoup d'interactions avec la culture francophone ou les régions à l'ouest du Canada. Socialement, il est principalement entouré de personnes de son groupe ethnique, mais il est ouvert à la diversité culturelle, étant donné la composition démographique de sa région. Andrew est hétérosexuel et vit probablement dans une famille traditionnelle ou dans un couple, ce qui cadre avec la majorité des personnes de son orientation dans sa tranche d'âge et son environnement."
10,"Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 10 : Variables avec une moyenne significativement plus élevée que la moyenne de la population en général : ses_regionAtlantic Variables avec une moyenne significativement plus basse que la moyenne de la population en général : c(""dv_voteChoiceBQ"", ""ses_educUniv"", ""ses_languageFrench"", ""ses_regionOntario"", ""ses_regionQuebec"", ""ses_regionWest"") Ce cluster est composé de : Hommes: 59.5%, Immigrants: 18.9% Âge: 53 ans Revenus: Revenu Bas: 13.5 % Revenu Mid: 81.1 % Revenu High: 5.4 % Langues: En: 100%, Fr: 0%, Autres: 0% EducBHS: 29.7% EducPostHS: 40.5% EducUniv: 29.7% Ethnie (Blanc): 94.6% Orientation (Hétéro): 78.4%","Pour une personne vivant au Canada, et plus précisément dans la région atlantique, un prénom qui serait à la fois anglophone et courant pourrait être ""James"". Voici une brève description pour le persona 10 : **Nom :** James **Âge :** 53 ans **Région :** Atlantique du Canada **Profil démographique :** - **Genre :** Homme - **Immigrant :** Non (compte tenu du pourcentage faible d'immigrants dans ce cluster) - **Langue :** Parle uniquement anglais - **Éducation :** A terminé des études postsecondaires, mais pas nécessairement universitaires - **Revenu :** Principalement moyen, avec peu de probabilité d'être dans la tranche de revenu élevé - **Origine ethnique :** Majoritairement blanc - **Orientation sexuelle :** Principalement hétérosexuel **Caractéristiques supplémentaires :** - James a grandi et vit toujours dans la région atlantique du Canada, ce qui influence ses habitudes et ses préférences culturelles. - Il n'a pas tendance 

In [9]:
write.csv(description_clusters, "description_clusters.csv", row.names = FALSE)

In [10]:
saveRDS(description_clusters, file = "../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_clusters_description.rds")